## Queries - Quick look

Craig Lage - 01-Mar-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.bestEffort import BestEffortIsr
from lsst.summit.utils.plotting import plot
from lsst.summit.utils import quickSmooth
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

In [ ]:
butler = butlerUtils.makeDefaultLatissButler(embargo=True)

In [ ]:
dayObs = 20240321
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter, record.observation_type)


In [ ]:
record

In [ ]:
dataId = dict(day_obs=20240308, detector=0, seq_num=199)
bestEffort = BestEffortIsr(embargo=True)
bestEffort.getExposure(dataId)
dataType = 'quickLookExp'
exp = butler.get(dataType, dataId)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,10))
plot(exp, figure=fig, stretch='ccs')
[ax1, ax2] = fig.axes
ax1.set_title(f"AuxTel, 20240308 seqNum 199", fontsize = 24)
slices = [[1830, 0.45, 0.6], [500, 0.85, 1.0], [3500, 0.1, 0.25]]
for [y, xmin, xmax] in slices:
    ax1.axhline(y, xmin, xmax, color='yellow')
plt.savefig("/home/c/cslage/u/AuxTel/isr/Defects_1_20240308_199.png")

In [ ]:
slices = [[1830, 2100, 2200], [500, 3775, 3875], [3500, 575, 675]]
arr = exp.image.array
mask = exp.getMaskedImage().getArrays()[1]
fig, axs = plt.subplots(3,1, figsize=(10,10))
plt.suptitle("AuxTel, 20240308 seqNum 199 defect slices", fontsize = 18)
for i, [y, xmin, xmax] in enumerate(slices):
    axs[i].set_title(f"Slice at y={y}, from {xmin} to {xmax}")
    axs[i].plot(arr[y, xmin:xmax], marker='x', color='blue', label='Data')
    ax1 = axs[i].twinx()
    ax1.plot(mask[y, xmin:xmax], marker='+', color='red', label='Mask')
    axs[i].legend(loc='upper left')
    ax1.legend()
plt.savefig("/home/c/cslage/u/AuxTel/isr/Defects_3_20240308_199.png")

In [ ]:
maskedImage = exp.getMaskedImage()
maskPlane = maskedImage.mask.getPlaneBitMask('INTRP')
maskPlane

In [ ]:
maskedImage.mask.getMaskPlaneDict()

In [ ]:
maskedImage.mask.getPlaneBitMask?

In [ ]:
for j in range(1800, 2000):
    line = arr[j, 2000:2200]
    print(j, np.max(line))

In [ ]:
plt.plot(arr[1830, 2100:2200], marker = 'x')

In [ ]:
for j in range(500,1000):
    line = arr[j, 3600:3900]
    print(j, np.max(line))

In [ ]:
plt.plot(arr[500, 3775:3875], marker = 'x')

In [ ]:
for j in range(3000, 4000):
    line = arr[j, 500:700]
    print(j, np.max(line))

In [ ]:
plt.plot(arr[3500, 575:675], marker = 'x')

In [ ]:
from numpy import unravel_index
unravel_index(arr.argmax(), arr.shape)

In [ ]:
donut = arr[1800:2100,2100:2400]
donut.max()

In [ ]:
_ = plot(donut, stretch='log')

In [ ]:
[ax1, ax2] = fig.axes

In [ ]:
ax_list

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure


In [ ]:
%matplotlib inline

fig = plotExp(exp)
[ax1, ax2] = fig.axes
ax1.text(2000, 2000, "Test", fontsize = 36, color='red')
slices = [[1830, 2100, 3000], [500, 3775, 3875], [3500, 575, 675]]
slices = [[1830, 0.45, 0.6], [500, 0.85, 1.0], [3500, 0.1, 0.25]]
for [y, xmin, xmax] in slices:
    ax1.axhline(y, xmin, xmax, color='yellow')
